In [10]:
# Header
from nupack import *
import numpy as np
config.parallelism = True

Model1 = Model(material='dna04', ensemble='some-nupack3', celsius=29, sodium=0.1, magnesium=0.0125) #Define model used for NUPACK calculations

In [11]:
# Import strands
Strands_list = []
Interfering = open("Designs.txt")
for line in Interfering:
    Strands_list.append(line.rstrip())
Interfering.close()
Nlength = 35 #Number of bases of each interfering strand (fixed in our case)

In [12]:
# Define strands/complexes and tasks 
Invader_Strand = Strand('CGGAATAAGGCAAGATAAGAACAGATACATAGGACGGATAGAGAA', name='Invader')
Interfering_free_energy_list = []
results = open("Results.txt", 'a')
for i in range(len(Strands_list)):
    Interfering_Strand = Strand(Strands_list[i], name ='Interfering_Strand')
    Interfering_Invader_Complex = Complex([Interfering_Strand, Invader_Strand], name='Interfering_Invader_Complex')
    Interfering_Complex = Complex([Interfering_Strand], name='Interfering_Complex')
    Invader_Complex = Complex([Invader_Strand], name='Invader_Complex')
    InteractionTube = Tube({Invader_Strand: 20e-9, Interfering_Strand: 1e-6}, complexes=SetSpec(include=[Interfering_Invader_Complex, Interfering_Complex, Invader_Complex]), name='InteractionTube')
    my_results = tube_analysis([InteractionTube], model=Model1, compute=['mfe'])
    Interfering_free_energy_list.append(str(my_results[Interfering_Complex].free_energy))
    results.write(str(Interfering_Strand) + '\n' +str(my_results[Interfering_Invader_Complex].free_energy) +'\n'+ str(my_results[Interfering_Invader_Complex].mfe[0].structure) +'\n'+'\n')
results.close()

In [13]:
# Calculate Energies
Results_list = []

results = open("Results.txt")
for line in results:
    Results_list.append(line.rstrip())
results.close()

Energies_list=[]

for i in range(len(Results_list[1::4])):
    Energies_list.append(float(Results_list[1::4][i])-float(Interfering_free_energy_list[i])-my_results[Invader_Complex].free_energy)

Complexes_ddG = open("Complex_ddG.txt", 'a')

for i in range(len(Energies_list)):
    Complexes_ddG.write(str("{:.2f}".format(Energies_list[i]))+'\n'),
Complexes_ddG.close()

In [14]:
# BM features
Bound_BM_Invader = open("Bound_BM_Invader.txt", 'a')
Binding_Blocks_BM= open("Binding_Blocks_BM.txt", 'a')
Longest_Bound_BM = open("Longest_Bound_BM.txt", 'a')
Longest_Unbound_BM = open("Longest_Unbound_BM.txt", 'a')

for j in range(0,len(Strands_list)):
    bound_BM = 0
    Interfering_Strand = Strand(Strands_list[j], name ='Interfering_Strand')
    probability_matrix = pairs(strands=[Interfering_Strand, Invader_Strand], model=Model1)
    probability_array=probability_matrix.to_array()
    rev_struc_body = str(Results_list[2::4][j][Nlength+1:Nlength+26][::-1])
    partition =[]
    bound_unbound = (''.join(x + ('' if x == nxt else ',') for x, nxt in zip(rev_struc_body, rev_struc_body[1:] + rev_struc_body[-1])))
    partition = bound_unbound.split(',')
    splitted = []
    for i in range(len(partition)):
        if '.' in partition[i]:
            splitted.append(float(len(partition[i])))
        else:
            splitted.append(str(len(partition[i])))          

    ###############################################################################
    # Code for number of binding blocks
    bindingblocks = 0
    for i in range(len(splitted)):
        if type(splitted[i]) == str:
            bindingblocks+=1
    Binding_Blocks_BM.write(str("{:.2f}".format((bindingblocks))+"\n"))
    ##############################################################################
    ##############################################################################
    # Code for longest stretch of bound bases
    longestbound = 0
    boundstretch = 0
    position = 0
    bound_stretches=[]
    for i in range(len(splitted)):
        if type(splitted[i]) == str:
            bound_stretches.append(float(splitted[i]))
    if len(bound_stretches) > 0:
        for i in range(0,splitted.index(str(int(max(bound_stretches))))):
            position += int(splitted[i])
        for m in range(Nlength+25-int(position+max(bound_stretches)), Nlength+25-int(position)): 
        #It can make sense to use the bases left and/or right of the stretch, too. This would be inbluded by +/-1 in the range.
            boundstretch += (1-probability_array[m][m])
        Longest_Bound_BM.write(str("{:.2f}".format(boundstretch)+"\n"))
    else:
        Longest_Bound_BM.write(str("{:.2f}".format((0))+"\n"))
    #############################################################################
    
    ##############################################################################
    # Code for longest stretch of unbound bases
    longestunbound = 0
    unboundstretch = 0
    position = 0
    unbound_stretches=[]
    for i in range(len(splitted)):
        if type(splitted[i]) == float:
            unbound_stretches.append(float(splitted[i]))
    if len(unbound_stretches) > 0:
        for i in range(0,splitted.index(max(unbound_stretches))):
            position += float(splitted[i])
        for m in range(Nlength+25-int(position+max(unbound_stretches)), Nlength+25-int(position)):
        #It can make sense to use the bases left and/or right of the stretch, too. This would be inbluded by +/-1 in the range.
            unboundstretch += probability_array[m][m]
        Longest_Unbound_BM.write(str("{:.2f}".format(unboundstretch)+"\n"))
    else:
        Longest_Unbound_BM.write(str("{:.2f}".format((0))+"\n"))
    #############################################################################
    bound = 0
    for m in range(Nlength,Nlength+25):
        bound += (1-probability_array[m][m])
    bound_BM += bound
    Bound_BM_Invader.write(str("{:.2f}".format((bound_BM))+"\n"))
Bound_BM_Invader.close()
Binding_Blocks_BM.close()
Longest_Bound_BM.close()
Longest_Unbound_BM.close()

In [25]:
# toe
Unbound_toe_Invader = open("Unbound_toe_Invader.txt", 'a')
Binding_Blocks_toe = open("Binding_Blocks_toe.txt", 'a')
Longest_Bound_toe = open("Longest_Bound_toe.txt", 'a')
Longest_Unbound_toe = open("Longest_Unbound_toe.txt", 'a')
toe_unbound_beginning = open("toe_unbound_beginning.txt", 'a')
for j in range(len(Strands_list)):
    unbound_toe = 0
    partition =[]
    Designed_Strand = Strand(Strands_list[j], name ='Designed_Strand')
    probability_matrix = pairs(strands=[Designed_Strand, Invader_Strand], model=Model1)
    probability_array=probability_matrix.to_array()
    rev_struc_toe = str(Results_list[2::4][j][Nlength+26:])[::-1]
    bound_unbound = (''.join(x + ('' if x == nxt else ',') for x, nxt in zip(rev_struc_toe, rev_struc_toe[1:] + rev_struc_toe[-1])))
    partition = bound_unbound.split(',')
    splitted = []
    for i in range(len(partition)):
        if '.' in partition[i]:
            splitted.append(float(len(partition[i])))
        else:
            splitted.append(str(len(partition[i])))          
    
    ###############################################################################
    # Code for number of binding blocks
    toe_bindingblocks = 0
    for i in range(len(splitted)):
        if type(splitted[i]) == str:
            toe_bindingblocks+=1
    Binding_Blocks_toe.write(str("{:.2f}".format((toe_bindingblocks))+"\n"))
    ##############################################################################
    
    ##############################################################################
    # Code for longest stretch of bound bases
    toe_longestbound = 0
    toe_boundstretch = 0
    position = 0
    toe_bound_stretches=[]
    
    for i in range(len(splitted)):
        if type(splitted[i]) == str:
            toe_bound_stretches.append(float(splitted[i]))
    if len(toe_bound_stretches) > 0:
        for i in range(0,splitted.index(str(int(max(toe_bound_stretches))))):
            position += int(splitted[i])
        for m in range(Nlength+45-int(position+max(toe_bound_stretches)), Nlength+45-int(position)):
        #It can make sense to use the bases left and/or right of the stretch, too. This would be inbluded by +/-1 in the range.
            toe_boundstretch += (1-probability_array[m][m])
        Longest_Bound_toe.write(str("{:.2f}".format(toe_boundstretch)+"\n"))
    else:
        Longest_Bound_toe.write(str("{:.2f}".format((0))+"\n"))
        
        
    #############################################################################

    ##############################################################################
    # Code for longest stretch of unbound bases
    toe_longestunbound = 0
    toe_unboundstretch = 0
    position = 0
    toe_unbound_stretches=[]
    for i in range(len(splitted)):
        if type(splitted[i]) == float:
            toe_unbound_stretches.append(splitted[i])
    if len(toe_unbound_stretches) > 0:
        for i in range(0,splitted.index(max(toe_unbound_stretches))):
            position += float(splitted[i])
        for m in range(Nlength+45-int(position+max(toe_unbound_stretches)), Nlength+45-int(position)):
        #It can make sense to use the bases left and/or right of the stretch, too. This would be inbluded by +/-1 in the range.
            toe_unboundstretch += probability_array[m][m]
        Longest_Unbound_toe.write(str("{:.2f}".format(toe_unboundstretch)+"\n"))
    else:
        Longest_Unbound_toe.write(str("{:.2f}".format((0))+"\n"))
    #############################################################################
    
    ##############################################################################
    # Code for unbound at beginning
    toe_unbound_beginning_count = 0
    if type(splitted[0]) == float:
        for i in range(Nlength+45-int(splitted[0]), Nlength+45):
            toe_unbound_beginning_count+= probability_array[i][i]
    else:
        toe_unbound_beginning_count = 0
    toe_unbound_beginning.write(str("{:.2f}".format((toe_unbound_beginning_count))+"\n"))
    #############################################################################
    
    bound_toe = 0
    bound = 0
    for m in range(Nlength+25,Nlength+45):
        bound += (1-probability_array[m][m])
    bound_toe += bound
    Unbound_toe_Invader.write(str("{:.2f}".format((20-bound_toe))+"\n"))
    
Unbound_toe_Invader.close()
Binding_Blocks_toe.close()
Longest_Bound_toe.close()
Longest_Unbound_toe.close()
toe_unbound_beginning.close()

In [28]:
# whole sequence
Binding_Blocks_sequence = open("Binding_Blocks_sequence.txt", 'a')
Transition_Area_BM = open("Transition_Area_BM.txt", 'a')
Transition_Area_toe = open("Transition_Area_toe.txt", 'a')

for j in range(len(Strands_list)):
    partition =[]
    Designed_Strand = Strand(Strands_list[j], name ='Designed_Strand')
    probability_matrix = pairs(strands=[Designed_Strand, Invader_Strand], model=Model1)
    probability_array=probability_matrix.to_array()
    rev_struc_toe = str(Results_list[2::4][j][Nlength+1:])[::-1]
    bound_unbound = (''.join(x + ('' if x == nxt else ',') for x, nxt in zip(rev_struc_toe, rev_struc_toe[1:] + rev_struc_toe[-1])))
    partition = bound_unbound.split(',')
    splitted = []
    
    for i in range(len(partition)):
        if '.' in partition[i]:
            splitted.append(float(len(partition[i])))
        else:
            splitted.append(str(len(partition[i])))          
    ###############################################################################
    # Code for number of binding blocks
    bindingblocks = 0
    for i in range(len(splitted)):
        if type(splitted[i]) == str:
            bindingblocks+=1
    Binding_Blocks_sequence.write(str("{:.2f}".format((bindingblocks))+"\n"))
    ##############################################################################
    
    ###############################################################################
    # Code for transition area
    sum_bases = 0
    position = 0
    transition_BM = 0
    transition_toe = 0
    for i in range(len(splitted)):
        sum_bases += float(splitted[i])
        if sum_bases > 20:
            if type(splitted[i]) == str:
                length=splitted[i]
                for m in range(Nlength+45-int(sum_bases), Nlength+25):
                    transition_BM += (1-probability_array[m][m])
                Transition_Area_BM.write(str("{:.2f}".format((float(transition_BM)))+"\n"))
                for m in range(Nlength+25,Nlength+25+int(length)-(int(sum_bases)-20)):
                    transition_toe += (1-probability_array[m][m])
                Transition_Area_toe.write(str("{:.2f}".format((float(transition_toe)))+"\n"))
                break
            else:
                Transition_Area_BM.write(str("{:.2f}".format((0))+"\n"))
                Transition_Area_toe.write(str("{:.2f}".format((0))+"\n"))
                break
            if type(splitted[i]) != str:
                print("No bases bound in transition area")
    ##############################################################################
    
    
Binding_Blocks_sequence.close()
Transition_Area_BM.close()
Transition_Area_toe.close()

In [31]:
###################################################################################
# Counts for different weak or strong bound/unbound bases
weak_BM = 15
strong_BM = 10
weak_toe = 12
strong_toe = 8


bound_weak_toe_dat = open("bound_weak_toe_dat.txt", 'a')
free_weak_toe_dat = open("free_weak_toe_dat.txt", 'a')
bound_strong_toe_dat = open("bound_strong_toe_dat.txt", 'a')
free_strong_toe_dat = open("free_strong_toe_dat.txt", 'a')
bound_weak_BM_dat = open("bound_weak_BM_dat.txt", 'a')
free_weak_BM_dat = open("free_weak_BM_dat.txt", 'a')
bound_strong_BM_dat = open("bound_strong_BM_dat.txt", 'a')
free_strong_BM_dat = open("free_strong_BM_dat.txt", 'a')



for j in range(len(Strands_list)):
    unbound = 0
    c=0
    partition =[]
    Designed_Strand = Strand(Strands_list[j], name ='Designed_Strand')
    probability_matrix = pairs(strands=[Designed_Strand, Invader_Strand], model=Model1)
    probability_array=probability_matrix.to_array()
    rev_struc_toe = str(Results_list[2::4][j][Nlength+1:])[::-1]
    bound_unbound = (''.join(x + ('' if x == nxt else ',') for x, nxt in zip(rev_struc_toe, rev_struc_toe[1:] + rev_struc_toe[-1])))
    partition = bound_unbound.split(',')
    splitted = []
    splitarray = []
    for i in range(len(partition)):
        splitarray.append(list(partition[i]))
    flat_splitarray = [x for xs in splitarray for x in xs]
    flat_splitarray.reverse()
    sequence_list = list("CGGAATAAGGCAAGATAAGAACAGATACATAGGACGGATAGAGAA")
    
    free_weak_BM = 0
    free_strong_BM = 0
    bound_strong_BM = 0
    bound_weak_BM = 0
    
    for i in range(25):
        if sequence_list[i] == 'A' or sequence_list[i] == 'T':
            free_weak_BM += probability_array[Nlength+i][Nlength+i]
            bound_weak_BM += (1-probability_array[Nlength+i][Nlength+i])
        elif sequence_list[i] == 'G' or sequence_list[i] == 'C':
            free_strong_BM += probability_array[Nlength+i][Nlength+i]
            bound_strong_BM += (1-probability_array[Nlength+i][Nlength+i])
            
    bound_weak_BM_dat.write(str("{:.2f}".format((bound_weak_BM))+"\n"))
    free_weak_BM_dat.write(str("{:.2f}".format((free_weak_BM))+"\n"))
    bound_strong_BM_dat.write(str("{:.2f}".format((bound_strong_BM))+"\n"))
    free_strong_BM_dat.write(str("{:.2f}".format((free_strong_BM))+"\n"))
    
    free_weak_toe = 0
    free_strong_toe = 0
    bound_weak_toe = 0
    bound_strong_toe = 0
    
    for i in range(25,45):
        if sequence_list[i] == 'A' or sequence_list[i] == 'T':
            free_weak_toe += probability_array[Nlength+i][Nlength+i]
            bound_weak_toe += (1-probability_array[Nlength+i][Nlength+i])
        elif sequence_list[i] == 'G' or sequence_list[i] == 'C':
            free_strong_toe += probability_array[Nlength+i][Nlength+i]
            bound_strong_toe += (1-probability_array[Nlength+i][Nlength+i])

    bound_weak_toe_dat.write(str("{:.2f}".format((bound_weak_toe))+"\n"))
    free_weak_toe_dat.write(str("{:.2f}".format((free_weak_toe))+"\n"))
    bound_strong_toe_dat.write(str("{:.2f}".format((bound_strong_toe))+"\n"))
    free_strong_toe_dat.write(str("{:.2f}".format((free_strong_toe))+"\n"))

bound_weak_toe_dat.close()
free_weak_toe_dat.close()
bound_strong_toe_dat.close()
free_strong_toe_dat.close()
bound_weak_BM_dat.close()
free_weak_BM_dat.close()
bound_strong_BM_dat.close()
free_strong_BM_dat.close()